In [ ]:
!pip install transformers datasets sentencepiece accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 21.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.7 MB/s eta 0:00:00


In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments
from datasets import load_dataset
import torch

In [ ]:
model_name = "lmqg/t5-base-squad-qag"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/2.36k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/20.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/1.51k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/892M [00:00<?, ?B/s]

In [ ]:
# Load and prepare dataset
dataset = load_dataset("lmqg/qag_squad")

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [ ]:
def preprocess_data(examples):
    # Preprocess input (context) and target (question and answer pair) for the model
    model_inputs = tokenizer(examples['paragraph'], max_length=512, truncation=True, padding="max_length")

    # Set up the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples['questions_answers'], max_length=128, truncation=True, padding="max_length")

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_datasets = dataset.map(preprocess_data, batched=True)

Map:   0%|          | 0/16462 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3856: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/2067 [00:00<?, ? examples/s]

Map:   0%|          | 0/2429 [00:00<?, ? examples/s]

In [ ]:
# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",             # Output directory for model checkpoints
    evaluation_strategy="epoch",        # Evaluate each epoch
    learning_rate=2e-5,                 # Learning rate
    per_device_train_batch_size=8,      # Batch size per device during training
    per_device_eval_batch_size=8,       # Batch size for evaluation
    num_train_epochs=3,                 # Number of training epochs
    weight_decay=0.01,                  # Weight decay
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
)

In [ ]:
# Train the model
trainer.train()

Epoch,Training Loss,Validation Loss
1,0.701500,0.903091
2,0.683900,0.897602
3,0.670100,0.899184


TrainOutput(global_step=6174, training_loss=0.6885131400960682, metrics={'train_runtime': 8987.1601, 'train_samples_per_second': 5.495, 'train_steps_per_second': 0.687, 'total_flos': 3.007399438319616e+16, 'train_loss': 0.6885131400960682, 'epoch': 3.0})

In [ ]:
def generate_question(context, model, tokenizer, max_length=512):
    # Tokenize the input context
    input_ids = tokenizer.encode(context, return_tensors="pt", max_length=max_length, truncation=True)

    # Move the input tensor to the device where the model is located
    input_ids = input_ids.to(model.device)

    # Generate outputs using the model
    output_sequences = model.generate(
        input_ids=input_ids,
        max_length=50,
        num_return_sequences=1,
        no_repeat_ngram_size=2,
        early_stopping=True
    )

    # Decode the output sequences to text
    questions = [tokenizer.decode(generated_sequence, skip_special_tokens=True) for generated_sequence in output_sequences]
    return questions[0]


In [ ]:
context = "Conducting PolymersPolymers, particularly those with a conjugated p-bond structure often show higher conductivity when doped with conductive materials.But the use of conductive polymers is limited since they have poor mechanical strength.Hence, a combination of mechanical and electrical properties can only find good applications in conductive polymers area.Sometimes, in a polymer blend, a bifunctional linker is doped to increase the conductivity of Polyaniline (PANI) (having conductivity) and polycaprolactum (PCL) (having mechanical strength) blend.Conductive polymers can be made using simple procedures like melt blending, solution blending etc., and can be used for antistatic and electromagnetic shielding applications."
question = generate_question(context, model, tokenizer)
print("Generated Question:", question)

Generated Question: question: What do polymers with conjugated p-bond structure show when doped with conductive materials?, answer: higher conductivity | question | Question: Why are a lot of ductile polymer used?,


In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
model.push_to_hub("Qgen_model_ACM")

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/abhir00p/Qgen_model_ACM/commit/85deb19564d2c5bc65a439cf29953b984d58c7ac', commit_message='Upload T5ForConditionalGeneration', commit_description='', oid='85deb19564d2c5bc65a439cf29953b984d58c7ac', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
tokenizer.push_to_hub("Qgen_model_ACM")

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/abhir00p/Qgen_model_ACM/commit/135183d30a96ec8ef0d6dee85544b8787b1bd403', commit_message='Upload tokenizer', commit_description='', oid='135183d30a96ec8ef0d6dee85544b8787b1bd403', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
from datasets import load_metric

# Load a metric for evaluation, e.g., BLEU, ROUGE
bleu = load_metric("bleu")

# Example to calculate BLEU for one instance
ref_question = "What do polymers with conjugated p-bond structure show when doped with conductive materials?."
gen_question = generate_question(context, model, tokenizer)

# BLEU expects a list of references and a list of hypotheses
bleu_score = bleu.compute(predictions=[gen_question.split()], references=[[ref_question.split()]])
print("BLEU Score:", bleu_score["bleu"])

/usr/local/lib/python3.10/dist-packages/datasets/load.py:752: FutureWarning: The repository for bleu contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.16.0/metrics/bleu/bleu.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:418: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(


BLEU Score: 0.3799538284139645
